In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')  # 윈도우용


## 📋 미션 2: 연령대별 구매 분석 (t-검정 마스터하기!) 👥

### 🎯 학습 목표

t-검정의 다양한 종류를 실전에서 완벽하게 활용하기

### 📖 배경 스토리

온라인 쇼핑몰에서 "연령대에 따라 구매 패턴이 다를까?" 하는 궁금증이 생겼습니다.

In [6]:
# 온라인 쇼핑몰의 구매 데이터
np.random.seed(789)
ages = np.random.randint(20, 60, 100)
purchase_amounts = 50 + 2*ages + np.random.normal(0, 20, 100)

print("📊 구매 데이터 미리보기")
print(f"연령 범위: {ages.min()}세 ~ {ages.max()}세")
print(f"구매금액 범위: {purchase_amounts.min():.0f}만원 ~ {purchase_amounts.max():.0f}만원")
print(f"평균 구매금액: {purchase_amounts.mean():.1f}만원")


📊 구매 데이터 미리보기
연령 범위: 20세 ~ 59세
구매금액 범위: 39만원 ~ 208만원
평균 구매금액: 130.1만원


### ✅ **문제 2-1: 기본 통계량 계산하기** 🎯

**🤔 해야할 일:** 일표본 t-검정을 위한 기본 통계량을 계산해보세요.

In [7]:
# TODO: 표본 평균 계산
sample_mean = purchase_amounts.mean()

# TODO: 표본 표준편차 계산
sample_std = purchase_amounts.std(ddof=1) 

# TODO: 표본 크기 계산
n =  len(purchase_amounts)

# TODO: 비교할 기준값 설정 (업계 기준 150만원)
standard_value = 150

print(f"표본 평균: {sample_mean:.2f}만원")
print(f"표본 표준편차: {sample_std:.2f}만원")
print(f"표본 크기: {n}명")
print(f"비교 기준값: {standard_value}만원")


표본 평균: 130.09만원
표본 표준편차: 33.90만원
표본 크기: 100명
비교 기준값: 150만원


### ✅ **문제 2-2: t-통계량 직접 계산하기** 🧮

**🤔 해야할 일:** t-검정의 t값을 직접 계산해보세요.

In [8]:
# TODO: t-통계량 공식 사용하기
# 힌트: t = (표본평균 - 기준값) / (표준편차 / sqrt(n))
t_stat = (sample_mean - standard_value) / (sample_std/np.sqrt(n))

print(f"직접 계산한 t-통계량: {t_stat:.3f}")

# 비교를 위해 scipy로도 계산해보기
from scipy.stats import ttest_1samp
t_stat_scipy, p_value = ttest_1samp(purchase_amounts, standard_value)
print(f"scipy로 계산한 t-통계량: {t_stat_scipy:.3f}")
print(f"p-value: {p_value:.4f}")


직접 계산한 t-통계량: -5.874
scipy로 계산한 t-통계량: -5.874
p-value: 0.0000


✅ t-통계량 -5.874
t-값이 음수라는 건,
현재 표본평균이 기준값보다 작다는 뜻이에요.

t의 절댓값이 클수록 기준값과 멀리 떨어져 있음을 의미합니다.

-5.874는 상당히 멀다는 뜻이에요 → 강한 증거!

✅ p-value 0.0000 (실제론 0.00000... 아주 작음)
이건 "이런 차이가 우연히 발생할 확률"을 말해요.

p < 0.05니까 → 우리는 통계적으로 유의미한 차이라고 말할 수 있어요.

### ✅ **문제 2-3: 결과 해석하기** 🎯

**🤔 해야할 일:** p-value를 보고 결과를 해석해보세요.

In [9]:
# TODO: 유의수준 설정
alpha = 0.05

print("🎯 결과 해석")

# TODO: p-value와 alpha 비교하기
if p_value < alpha:
    print(f"✅ p-value ({p_value:.4f}) < α ({alpha})")
    print("→ 귀무가설 기각! 업계 기준과 유의하게 다릅니다.")

    # TODO: 평균이 더 큰지 작은지 확인하기
    if sample_mean > standard_value:
        print("→ 우리 고객들이 업계 평균보다 더 많이 구매합니다! 🎉")
    else:
        print("→ 우리 고객들이 업계 평균보다 적게 구매합니다. 😅")
else:
    print(f"❌ p-value ({p_value:.4f}) ≥ α ({alpha})")
    print("→ 업계 기준과 유의한 차이가 없습니다.")


🎯 결과 해석
✅ p-value (0.0000) < α (0.05)
→ 귀무가설 기각! 업계 기준과 유의하게 다릅니다.
→ 우리 고객들이 업계 평균보다 적게 구매합니다. 😅


### ✅ **문제 2-4: 연령대별 그룹 나누기** 👥

**🤔 해야할 일:** 독립표본 t-검정을 위해 데이터를 두 그룹으로 나누세요.

In [10]:
# TODO: 젊은층 데이터 필터링 (20-39세)
young_mask = (ages >= 20) & (ages <= 39) 
group1_purchases = purchase_amounts[young_mask]

# TODO: 중년층 데이터 필터링 (40-59세)
middle_mask = (ages >= 40) & (ages <= 59)
group2_purchases = purchase_amounts[middle_mask]

print("👥 그룹 나누기 결과")
print(f"젊은층 (20-39세): {len(group1_purchases)}명")
print(f"중년층 (40-59세): {len(group2_purchases)}명")


👥 그룹 나누기 결과
젊은층 (20-39세): 47명
중년층 (40-59세): 53명


### ✅ **문제 2-5: 그룹별 기술통계 계산하기** 📊

**🤔 해야할 일:** 각 그룹의 평균과 표준편차를 계산해보세요.

In [13]:
# TODO: 각 그룹의 평균 계산
group1_mean = group1_purchases.mean()
group2_mean = group2_purchases.mean()

# TODO: 각 그룹의 표준편차 계산
group1_std = group1_purchases.std()
group2_std = group2_purchases.std()

print("📊 그룹별 기술통계")
print(f"젊은층 평균: {group1_mean:.1f}만원 (표준편차: {group1_std:.1f})")
print(f"중년층 평균: {group2_mean:.1f}만원 (표준편차: {group2_std:.1f})")

# TODO: 평균 차이 계산
diff = group2_mean - group1_mean
print(f"평균 차이: {diff:.1f}만원")


📊 그룹별 기술통계
젊은층 평균: 106.4만원 (표준편차: 25.0)
중년층 평균: 151.1만원 (표준편차: 25.6)
평균 차이: 44.6만원


### ✅ **문제 2-6: 독립표본 t-검정 실행하기** 🧪

**🤔 해야할 일:** 두 그룹 간의 차이를 t-검정으로 확인해보세요.

In [15]:
from scipy.stats import ttest_ind

# TODO: 독립표본 t-검정 실행
t_stat, p_value = ttest_ind(group1_purchases, group2_purchases)

print("🧮 t-검정 결과")
print(f"t-통계량: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")

# TODO: 결과 해석하기
alpha = 0.05
print("🎯 비즈니스 해석")

if p_value < alpha:
    print("✅ 통계적으로 유의한 차이가 있습니다!")

    if group2_mean > group1_mean :
        print("→ 중년층이 젊은층보다 더 많이 구매합니다!")
    else:
        print("→ 젊은층이 중년층보다 더 많이 구매합니다!")
else:
    print("❌ 통계적으로 유의한 차이가 없습니다.")


🧮 t-검정 결과
t-통계량: -8.709
p-value: 0.0000
🎯 비즈니스 해석
✅ 통계적으로 유의한 차이가 있습니다!
→ 중년층이 젊은층보다 더 많이 구매합니다!


### ✅ **문제 2-7: 등분산성 검정하기** 🔍

**🤔 해야할 일:** 두 그룹의 분산이 같은지 확인해보세요.

In [16]:
from scipy.stats import levene

# TODO: 분산 직접 비교하기
var1 = group1_purchases.var(ddof=1)
var2 = group2_purchases.var(ddof=1)

print("📊 분산 비교")
print(f"젊은층 분산: {var1:.1f}")
print(f"중년층 분산: {var2:.1f}")

# TODO: Levene 검정 실행하기
levene_stat, levene_p = levene(group1_purchases, group2_purchases)

print("🧪 Levene 등분산성 검정")
print(f"p-value: {levene_p:.4f}")

# TODO: 등분산성 판단하기
if levene_p >= 0.05 :
    print("✅ 등분산성 가정 만족")
else:
    print("❌ 등분산성 가정 위배")


📊 분산 비교
젊은층 분산: 637.1
중년층 분산: 669.5
🧪 Levene 등분산성 검정
p-value: 0.8106
✅ 등분산성 가정 만족
